In [963]:
import gurobipy as gb
from gurobipy import *

In [964]:
prob = gb.Model("Portfolio Optimization")

In [965]:
import pandas as pd
import numpy as np

In [966]:
df = pd.read_excel('/Users/matthewbuttlerives/Desktop/Copy of Markowitz.xlsx', sheet_name='Matthew', index_col = 0)

In [967]:
df.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
Weekly Adjusted Closing Prices,,,,,,,
Timestamp,POLYGON,CARDANO,ACUITY,HD,JPM,TD,VISA
2023-02-19 00:00:00,1.22,0.3564,2.21,296.66,140.93,91.11,219.55
2023-02-12 00:00:00,1.485,0.4061,2.38,317.95,142.24,92.87,223.56
2023-02-05 00:00:00,1.25,0.3685,2.31,316.54,141.04,93.01,227.2
2023-01-29 00:00:00,1.251,0.3991,2.46,331.5,141.09,92.42,230.13


In [968]:
df = df.rename(columns=df.iloc[0])
df = df.iloc[1: , :]

In [969]:
df.head()

,POLYGON,CARDANO,ACUITY,HD,JPM,TD,VISA
Weekly Adjusted Closing Prices,,,,,,,
2023-02-19 00:00:00,1.22,0.3564,2.21,296.66,140.93,91.11,219.55
2023-02-12 00:00:00,1.485,0.4061,2.38,317.95,142.24,92.87,223.56
2023-02-05 00:00:00,1.25,0.3685,2.31,316.54,141.04,93.01,227.2
2023-01-29 00:00:00,1.251,0.3991,2.46,331.5,141.09,92.42,230.13
2023-01-22 00:00:00,1.145,0.3822,2.35,316.69,140.32,91.42,231.44


In [970]:
df = df.pct_change(-1)
df = df.dropna()

In [971]:
df.head()

,POLYGON,CARDANO,ACUITY,HD,JPM,TD,VISA
Weekly Adjusted Closing Prices,,,,,,,
2023-02-19 00:00:00,-0.178451,-0.122384,-0.071429,-0.066960,-0.009210,-0.018951,-0.017937
2023-02-12 00:00:00,0.188000,0.102035,0.030303,0.004454,0.008508,-0.001505,-0.016021
2023-02-05 00:00:00,-0.000799,-0.076673,-0.060976,-0.045128,-0.000354,0.006384,-0.012732
2023-01-29 00:00:00,0.092576,0.044218,0.046809,0.046765,0.005487,0.010939,-0.005660
2023-01-22 00:00:00,0.157735,0.036615,0.000000,0.005365,0.038792,0.025118,0.031786


In [972]:
std_assets = df.std()
df.std()

POLYGON    0.172050
CARDANO    0.117051
ACUITY     0.089980
HD         0.042385
JPM        0.041704
TD         0.023841
VISA       0.039605
dtype: float64

In [973]:
mean_assets = df.mean()
df.mean()

POLYGON    0.009421
CARDANO   -0.010493
ACUITY    -0.004891
HD        -0.000865
JPM        0.001773
TD        -0.001256
VISA       0.002558
dtype: float64

In [974]:
mean_assets[0]

0.009421295843970052

In [975]:
df.corr()

,POLYGON,CARDANO,ACUITY,HD,JPM,TD,VISA
POLYGON,1.000000,0.672471,0.334001,0.227917,0.397068,0.125275,0.354783
CARDANO,0.672471,1.000000,0.456010,0.086803,0.314210,0.294104,0.306154
ACUITY,0.334001,0.456010,1.000000,0.327926,0.335869,0.308205,0.488938
HD,0.227917,0.086803,0.327926,1.000000,0.487609,0.350069,0.684064
JPM,0.397068,0.314210,0.335869,0.487609,1.000000,0.637351,0.563162
TD,0.125275,0.294104,0.308205,0.350069,0.637351,1.000000,0.357790
VISA,0.354783,0.306154,0.488938,0.684064,0.563162,0.357790,1.000000


In [976]:
# To create Objective function, we need to either determine if we are maximizing return or minimizing risk.
# For the first question, lets just minimize risk.

# We need then the expected return formula, which is each assets average return, multiplied by its decision variable
# Then we need the variance of that return, which is our risk.

# To determine variance, we take each assets and multiply their standard deviations, with their correlations, by 2

Stock_List = df.columns.values.tolist()
sl = len(Stock_List)

std_assets

POLYGON    0.172050
CARDANO    0.117051
ACUITY     0.089980
HD         0.042385
JPM        0.041704
TD         0.023841
VISA       0.039605
dtype: float64

In [977]:
cor = df.corr()
cor['VISA']

POLYGON    0.354783
CARDANO    0.306154
ACUITY     0.488938
HD         0.684064
JPM        0.563162
TD         0.357790
VISA       1.000000
Name: VISA, dtype: float64

In [978]:
x = prob.addVars(sl,lb = 0,ub = 1, vtype = GRB.CONTINUOUS,name = ['Investment Proportion in '+a for a in Stock_List])

In [979]:
part1 = sum(x[i]**2*std_assets[i]**2 for i in range(sl))

In [980]:
part1

<gurobi.QuadExpr: 0.0 + [ 0.029601040148346337 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.013701034423255735 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.008096429021711565 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0017965301786640074 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0017392636384829263 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0005684133569290863 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0015685201121514848 <gurobi.Var *Awaiting Model Update*> ^ 2 ]>

In [981]:
def assets_covaraince(Stocks,std,decision_variables,correlation):
    completed = []
    part2 = 0
    for stock in Stocks:
        position = Stocks.index(stock)
        part_ = sum(2*decision_variables[position]*decision_variables[j]*std[position]*std[j]*correlation[stock][j] for j in range(len(Stocks)) if j > position)
        completed.append(position)
        part2 = part2 + part_ 
    return part2      

In [982]:
part3 = assets_covaraince(Stock_List,std_assets,x,cor)

In [983]:
part4 = part1+part3

In [984]:
part4

<gurobi.QuadExpr: 0.0 + [ 0.029601040148346337 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.013701034423255735 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.008096429021711565 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0017965301786640074 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0017392636384829263 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0005684133569290863 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0015685201121514848 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.027085299880395274 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.010341354452465139 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.003324132104680774 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.005698116823940041 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0010277361432688916 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.004834948773913459 <guro

In [985]:
prob.setObjective(part4)

In [986]:
prob.addConstr(sum(x[i] for i in range(sl)) == 1)

prob.addConstr(x[0] >= 0.15)
prob.addConstr(x[0] <= 0.3)
prob.addConstr(x[5] >= 0.35)
prob.addConstr(x[2] >= 0.033)

#prob.addConstr(sum(x[i]*mean_assets[i] for i in range(sl)) >= 0.005)

<gurobi.Constr *Awaiting Model Update*>

In [987]:
#Finally, Optimize Code
prob.optimize()
prob.display()

print("The optimal risk is: ", prob.objval)

print("Optimal value for Dvs:")
for v in prob.getVars():
    print(v.varName, "=", round(v.X,3))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 7 columns and 11 nonzeros
Model fingerprint: 0x69164518
Model has 28 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-03, 6e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 7 columns, 7 nonzeros
Presolved model has 28 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 6
 AA' NZ     : 2.100e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.68117973e+05 -3.68155473e+05  6.50e+03 5.56e-06  4.00e+05     0s
   1   1.79750111e+

In [304]:
cor

,POLYGON,CARDANO,ACUITY,HD,JPM,TD,VISA
POLYGON,1.000000,0.672471,0.334001,0.227917,0.397068,0.125275,0.354783
CARDANO,0.672471,1.000000,0.456010,0.086803,0.314210,0.294104,0.306154
ACUITY,0.334001,0.456010,1.000000,0.327926,0.335869,0.308205,0.488938
HD,0.227917,0.086803,0.327926,1.000000,0.487609,0.350069,0.684064
JPM,0.397068,0.314210,0.335869,0.487609,1.000000,0.637351,0.563162
TD,0.125275,0.294104,0.308205,0.350069,0.637351,1.000000,0.357790
VISA,0.354783,0.306154,0.488938,0.684064,0.563162,0.357790,1.000000


In [354]:
check = pd.read_excel('/Users/matthewbuttlerives/Desktop/Copy of Markowitz.xlsx', sheet_name='Markowitz', index_col = 0)

In [355]:
check

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
Monthly Adjusted Closing Prices,,,,,
Timestamp,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
2016-06-01 00:00:00,5966.8,9606.71,17640.17,126.91,1149.3
2016-05-03 00:00:00,6230.8,10262.74,17787.2,129.05,1154.79
2016-04-01 00:00:00,6241.9,10038.97,17773.64,130.88,1130.85
2016-03-01 00:00:00,6174.9,9965.51,17685.09,128.64,1114.03
...,...,...,...,...,...
2011-04-01 00:00:00,6069.9,7514.46,12810.54,144.26,865.29
2011-03-01 00:00:00,5908.8,7041.31,12319.73,142.02,843.55
2011-02-01 00:00:00,5994,7272.32,12226.34,149.11,823.45


In [356]:
check = check.rename(columns=check.iloc[0])
check

,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
Monthly Adjusted Closing Prices,,,,,
Timestamp,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
2016-06-01 00:00:00,5966.8,9606.71,17640.17,126.91,1149.3
2016-05-03 00:00:00,6230.8,10262.74,17787.2,129.05,1154.79
2016-04-01 00:00:00,6241.9,10038.97,17773.64,130.88,1130.85
2016-03-01 00:00:00,6174.9,9965.51,17685.09,128.64,1114.03
...,...,...,...,...,...
2011-04-01 00:00:00,6069.9,7514.46,12810.54,144.26,865.29
2011-03-01 00:00:00,5908.8,7041.31,12319.73,142.02,843.55
2011-02-01 00:00:00,5994,7272.32,12226.34,149.11,823.45


In [357]:
check = check.iloc[1: , :]
check

,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
Monthly Adjusted Closing Prices,,,,,
2016-06-01,5966.8,9606.71,17640.17,126.91,1149.3
2016-05-03,6230.8,10262.74,17787.2,129.05,1154.79
2016-04-01,6241.9,10038.97,17773.64,130.88,1130.85
2016-03-01,6174.9,9965.51,17685.09,128.64,1114.03
2016-02-01,6097.1,9495.4,16516.5,119.01,1033.9
...,...,...,...,...,...
2011-04-01,6069.9,7514.46,12810.54,144.26,865.29
2011-03-01,5908.8,7041.31,12319.73,142.02,843.55
2011-02-01,5994,7272.32,12226.34,149.11,823.45


In [358]:
check = check.pct_change(-1)
check = check.dropna()

In [359]:
check

,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
Monthly Adjusted Closing Prices,,,,,
2016-06-01,-0.042370,-0.063923,-0.008266,-0.016583,-0.004754
2016-05-03,-0.001778,0.022290,0.000763,-0.013982,0.021170
2016-04-01,0.010850,0.007371,0.005007,0.017413,0.015098
2016-03-01,0.012760,0.049509,0.070753,0.080918,0.077503
2016-02-01,0.002186,-0.030895,0.003049,-0.039002,-0.001429
...,...,...,...,...,...
2011-05-02,-0.013163,-0.029379,-0.018793,-0.026757,-0.019635
2011-04-01,0.027264,0.067196,0.039839,0.015772,0.025772
2011-03-01,-0.014214,-0.031766,0.007638,-0.047549,0.024409


In [360]:
std_assets = check.std()
check.std()

FTSE 100              0.032549
DAX                   0.052861
DJIA                  0.032890
DJ Asian Titans 50    0.045223
Russell 2000          0.046334
dtype: float64

In [361]:
mean_assets = check.mean()
check.mean()

FTSE 100              0.000695
DAX                   0.006402
DJIA                  0.006931
DJ Asian Titans 50   -0.001075
Russell 2000          0.006875
dtype: float64

In [362]:
cor = check.corr()
check.corr()

,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
FTSE 100,1.000000,0.755267,0.772833,0.687467,0.649829
DAX,0.755267,1.000000,0.703816,0.668050,0.694259
DJIA,0.772833,0.703816,1.000000,0.721454,0.848499
DJ Asian Titans 50,0.687467,0.668050,0.721454,1.000000,0.670301
Russell 2000,0.649829,0.694259,0.848499,0.670301,1.000000


In [363]:
prob = gb.Model("Portfolio Optimization")
Stock_List = check.columns.values.tolist()
sl = len(Stock_List)
x = prob.addVars(sl,lb = 0,ub = 1, vtype = GRB.CONTINUOUS,name = ['Investment Proportion in '+a for a in Stock_List])

In [364]:
x

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

In [365]:
part1 = sum(x[i]**2*std_assets[i]**2 for i in range(sl))

In [366]:
part1

<gurobi.QuadExpr: 0.0 + [ 0.0010594275581240213 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0027942595019572725 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0010817514790623722 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.002045125328049484 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0021468511097900867 <gurobi.Var *Awaiting Model Update*> ^ 2 ]>

In [367]:
def assets_covaraince(Stocks,std,decision_variables,correlation):
    completed = []
    part2 = 0
    for stock in Stocks:
        position = Stocks.index(stock)
        part_ = sum(2*decision_variables[position]*decision_variables[j]*std[position]*std[j]*correlation[stock][j] for j in range(len(Stocks)) if j > position)
        completed.append(position)
        part2 = part2 + part_ 
    return part2

In [368]:
part3 = assets_covaraince(Stock_List,std_assets,x,cor)
part4 = part1 + part3

In [369]:
part4

<gurobi.QuadExpr: 0.0 + [ 0.0010594275581240213 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0027942595019572725 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0010817514790623722 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.002045125328049484 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0021468511097900867 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0025989589930001107 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0016546839066096906 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0020238473661027497 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0019600440013050374 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0024472947788645055 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0031939795355041185 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.003400838480356591 <gurobi.Var *Awaiting Model U

In [370]:
prob.setObjective(part4)

In [371]:
prob.addConstr(sum(x[i] for i in range(sl)) == 1)
#model.params.NonConvex = 2

<gurobi.Constr *Awaiting Model Update*>

In [372]:
#Finally, Optimize Code
prob.optimize()
prob.display()

print("The optimal risk is: ", prob.objval)

print("Optimal value for Dvs:")
for v in prob.getVars():
    print(v.varName, "=", round(v.X,3))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x1187641c
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-03, 7e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 1 rows, 5 columns, 5 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.47487649e+05 -2.47487649e+05  5.00e+03 9.16e-07  2.50e+05     0s
   1   2.93585435e-01 -3.15066610e+01  5.07e+00 9.29e-10  

In [1005]:
import gurobipy as gb
from gurobipy import *
prob = gb.Model("Portfolio Optimization")
import pandas as pd
import numpy as np
df = pd.read_excel('/Users/matthewbuttlerives/Desktop/Copy of Markowitz.xlsx', sheet_name='TFSA', index_col = 0)
df.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
Weekly Adjusted Closing Prices,,,,,,,
Timestamp,BCE,CAMECO,DOC,LOW,SBUX,TD,WEED
02/26/2023,60.2,37.39,0.195,207.09,102.56,66.77,3.16
02/19/2023,60.46,37.13,0.2,201.54,101.6,66.96,3.16
02/12/2023,61.78,38.78,0.225,212.75,107.1,68.91,3.37
02/05/2023,60.82,38.98,0.205,208.78,107.09,69.69,3.04


In [1006]:
df = df.rename(columns=df.iloc[0])
df = df.iloc[1: , :]
df.head()

,BCE,CAMECO,DOC,LOW,SBUX,TD,WEED
Weekly Adjusted Closing Prices,,,,,,,
02/26/2023,60.2,37.39,0.195,207.09,102.56,66.77,3.16
02/19/2023,60.46,37.13,0.2,201.54,101.6,66.96,3.16
02/12/2023,61.78,38.78,0.225,212.75,107.1,68.91,3.37
02/05/2023,60.82,38.98,0.205,208.78,107.09,69.69,3.04
01/29/2023,61.77,37.21,0.21,215.97,104.3,68.95,3.93


In [1007]:
df = df.pct_change(-1)
df = df.dropna()
df.head()

,BCE,CAMECO,DOC,LOW,SBUX,TD,WEED
Weekly Adjusted Closing Prices,,,,,,,
02/26/2023,-0.004300,0.007002,-0.025000,0.027538,0.009449,-0.002838,0.000000
02/19/2023,-0.021366,-0.042548,-0.111111,-0.052691,-0.051354,-0.028298,-0.062315
02/12/2023,0.015784,-0.005131,0.097561,0.019015,0.000093,-0.011192,0.108553
02/05/2023,-0.015380,0.047568,-0.023810,-0.033292,0.026750,0.010732,-0.226463
01/29/2023,-0.008825,0.011416,-0.045455,0.066571,-0.043295,0.004077,0.010283


In [1008]:
std_assets = df.std()
df.std()

BCE       0.024237
CAMECO    0.070735
DOC       0.097589
LOW       0.048907
SBUX      0.040888
TD        0.030188
WEED      0.132317
dtype: float64

In [1009]:
mean_assets = df.mean()
df.mean()

BCE      -0.002759
CAMECO    0.004057
DOC      -0.018459
LOW      -0.000222
SBUX      0.005043
TD       -0.002331
WEED     -0.008735
dtype: float64

In [1010]:
cor = df.corr()
df.corr()

,BCE,CAMECO,DOC,LOW,SBUX,TD,WEED
BCE,1.000000,0.249405,0.123750,0.298622,0.246923,0.339338,0.227255
CAMECO,0.249405,1.000000,0.306540,0.220090,0.182125,0.209184,0.185045
DOC,0.123750,0.306540,1.000000,0.313308,0.519731,0.394813,0.309156
LOW,0.298622,0.220090,0.313308,1.000000,0.334900,0.342774,0.491827
SBUX,0.246923,0.182125,0.519731,0.334900,1.000000,0.549908,0.301845
TD,0.339338,0.209184,0.394813,0.342774,0.549908,1.000000,0.392844
WEED,0.227255,0.185045,0.309156,0.491827,0.301845,0.392844,1.000000


In [1011]:
prob = gb.Model("Portfolio Optimization")
Stock_List = df.columns.values.tolist()
sl = len(Stock_List)
x = prob.addVars(sl,lb = 0,ub = 1, vtype = GRB.CONTINUOUS,name = ['Investment Proportion in '+a for a in Stock_List])

In [1012]:
x

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>}

In [1013]:
part1 = sum(x[i]**2*std_assets[i]**2 for i in range(sl))

In [1014]:
part1

<gurobi.QuadExpr: 0.0 + [ 0.0005874376979300652 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.005003391130647623 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.009523530884097225 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.002391860468666794 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0016718352487460782 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0009113428935419475 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.017507724313543097 <gurobi.Var *Awaiting Model Update*> ^ 2 ]>

In [1015]:
part3 = assets_covaraince(Stock_List,std_assets,x,cor)
part4 = part1 + part3

In [1016]:
part4

<gurobi.QuadExpr: 0.0 + [ 0.0005874376979300652 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.005003391130647623 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.009523530884097225 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.002391860468666794 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0016718352487460782 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0009113428935419475 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.017507724313543097 <gurobi.Var *Awaiting Model Update*> ^ 2 + 0.0008551627725890789 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0005854009412695161 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0007079475839428622 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0004894067318494441 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0004965748072305548 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.0014576053501652094 <

In [1017]:
prob.setObjective(part4)
prob.addConstr(sum(x[i] for i in range(sl)) == 1)
prob.addConstr(sum(x[i] for i in range(sl)) == 1)
prob.addConstr(x[2] >= 0.03)
prob.addConstr(x[5] >= 0.16)

#Finally, Optimize Code
prob.optimize()
prob.display()

print("The optimal risk is: ", prob.objval)

print("Optimal value for Dvs:")
for v in prob.getVars():
    print(v.varName, "=", round(v.X,3))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 7 columns and 16 nonzeros
Model fingerprint: 0x2198df64
Model has 28 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-03, 4e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 1e+00]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 7 columns, 7 nonzeros
Presolved model has 28 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 6
 AA' NZ     : 2.100e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.73463354e+05 -2.73463354e+05  6.50e+03 5.14e-06  2.53e+05     0s
   1   5.86543994e+